# Normalization pipeline

Die Quelldaten sind aus meinem ersten RAG-Projekt und wurden mit fruendlicher Genehmigung von einer Webseite gescrappt.

**Was zu tun ist:**
- Daten von Marketingaussagen und Firmengeschichte befreien
- Die Specs normalisieren

Das meiste werde ich mit einem LLM machen. Die Ausgangsdaten werden stichprobenartig kontrolliert. Ziel ist ein funktionierendes und korrektes retrival, welches nicht zwingend mit der realen Welt (korrekte technische Daten z.B.) übereinstimmen muss. In einem Realworldprojekt ist das selbstverständlich etwas anderes.

Wärend der Bereinigung werden die Daten in Specs und Descriptions aufgeteilt.

In [ ]:
import os
import json
import time
import pandas as pd
from tqdm import tqdm
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()

df = pd.read_json('./products_raw.json')

api_key = os.getenv('MISTRAL_API_KEY')
model = 'mistral-large-latest'
client = Mistral(api_key=api_key)

descs_promt = open('./description_agent.md', 'r').read()
specs_promt = open('./specs_agent.md', 'r').read()
summs_promt = open('./summary_agent.md').read()

descs_chunks = []
specs_chunks = []
summs_chunks = []

## Descriptions

Die Beschreibungen werden über ein LLM bereinigt. Der Systempromt liegt als md-File vor. 

### Funktionen

- **agent_request**: Allgemeine Retrivalfunktion 
- **format_spec**: Bereitet Embeddingdocs für Specs auf

In [37]:
def agent_request(promt, content):
    response = client.chat.complete(
        model = model,
        messages = [
            {
                'role': 'system',
                'content': promt
            },
            {
                'role': 'user',
                'content': content,
            }
        ],
        response_format = {"type": "json_object"}
    )

    return response

def format_spec(group_name, specs_list):
    if not specs_list:
        return ''
    
    if '-' in group_name:
        label, unit = group_name.rsplit('-', 1)
        header = f"{label} ({unit})"
    else:
        header = group_name
    
    spec_parts = []

    for key, value in specs_list.items():
        readable_key = key.replace('_', ' ')
        spec_parts.append(f"{readable_key}: {value}")

    return f"{header} - {', '.join(spec_parts)}"

# Erstellung der Chunks

Die Daten werden an ein LLM geliefert und wir erhalten nur die erzeugen Artefarkte:

```json
# Specs
{
  "Abmessungen-cm": {
    "Außenmaße_Breite": 67,
    "Außenmaße_Tiefe": 72,
    "Außenmaße_Höhe": 132
  },
  "Gewicht-kg": {
    "netto": 71,
    "brutto": 83
  },
  "Volumen-l": {
    "Kühlinhalt": 280
  },
  "Temperatur-celsius": {},
  "Leistung-watt": {},
  "Energieverbrauch-kwh": {},
  "Elektrisch": {},
  "Ausstattung": {},
  "Sonstiges": {}
}

# Beschreibung
[
  "Optimierter Absatz zu einem Aspekt des Produkts.",
  "Optimierter Absatz zu einem weiteren Aspekt des Produkts.",
  "Optimierter Absatz zu noch einem Aspekt des Produkts."
]

# Summary
{
  "summary": "Der Kirsch LABO-288 PRO-ACTIVE ist ein Laborkühlschrank mit 280 Litern Nutzvolumen und Temperaturregelung von 0-15°C. Mit statischer Belüftung, automatischer Abtauung und DIN 13221 Konformität ist er ideal für die sichere Lagerung in Labor und Medizin.",
  "category": "Laborkühlschrank",
  "manufacturer": "Kirsch"
}
```

Diese werden in des Schema der Chunks integriert:

In [ ]:
for index, row in tqdm(df.iterrows(), total=len(df)):

    summs_response = agent_request(summs_promt, json.dumps(row.to_dict()))
    # print(summs_response)
    # time.sleep(1)
    descs_response = agent_request(descs_promt, row['description'])
    # print(descs_response)
    # time.sleep(1)
    specs_response = agent_request(specs_promt, json.dumps(row['specs']))
    # print(specs_response)
    # time.sleep(1)

    summs_result = json.loads(summs_response.choices[0].message.content)                              
    descs_result = json.loads(descs_response.choices[0].message.content)
    specs_result = json.loads(specs_response.choices[0].message.content)

    category = summs_result['category']
    manufacturer = summs_result['manufacturer']

    summs_chunks.append({
        'document': summs_result['summary'],
        'metadata': {
            'chunk_type': 'description',
            'product_id': row['id'],
            'product_title': row['title'],
            'product_url': row['url'],
            'product_category': category,
            'product_manufacturer': manufacturer
        }
    })

    for praragraph in descs_result:
        descs_chunks.append({
            'document': praragraph,
            'metadata': {
                'chunk_type': 'description',
                'product_id': row['id'],
                'product_title': row['title'],
                'product_url': row['url'],
                'product_category': category,
                'product_manufacturer': manufacturer
            }
        })

    for group_name, specs_list in specs_result.items():
        if specs_list:
            specs_chunks.append({
                'document': format_spec(group_name, specs_list),
                'metadata': {
                    'chunk_type': 'description',
                    'product_id': row['id'],
                    'product_title': row['title'],
                    'product_url': row['url'],
                    'product_category': category,
                    'product_manufacturer': manufacturer
                }
            })

    # Testing only
    # if index == 4: break

 33%|███▎      | 50/152 [14:11<20:42, 12.18s/it]

## Quality Checks

Wir müssen zuletzt noch in die Chunks schauen ob sie die wichtigsten Rahmenbedinungen einhalten. Vor allem die Länge da die meisten Models eine beschränkte Anzahl an Tokes verarbeiten kann und die darf von den Chunks nicht überschritten werden.

In [61]:
def stats(df, name):

    length = df['document'].str.len()

    print(f'--- {name} ---')
    print(f"{length.describe()}")
    print(f'TOTAL CHUNKS {len(df)}')
    print('\n')


In [62]:
stats(pd.DataFrame(summs_chunks), 'Summs')
stats(pd.DataFrame(descs_chunks), 'Descriptions')
stats(pd.DataFrame(specs_chunks), 'Specs')

--- Summs ---
count      6.000000
mean     352.000000
std       63.545259
min      249.000000
25%      339.500000
50%      350.000000
75%      374.750000
max      444.000000
Name: document, dtype: float64
TOTAL CHUNKS 6


--- Descriptions ---
count     39.000000
mean     383.076923
std      109.593600
min      217.000000
25%      307.000000
50%      364.000000
75%      425.500000
max      738.000000
Name: document, dtype: float64
TOTAL CHUNKS 39


--- Specs ---
count     35.000000
mean     152.257143
std      198.847169
min       29.000000
25%       40.500000
50%       75.000000
75%      148.000000
max      839.000000
Name: document, dtype: float64
TOTAL CHUNKS 35




In [ ]:
pd.DataFrame(summs_chunks).to_json('summs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
pd.DataFrame(descs_chunks).to_json('descs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
pd.DataFrame(specs_chunks).to_json('specs_chunks.jsonl', orient='records', lines=True, force_ascii=False)